<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# Global Analysis Mode of FAST-OAD-GA

The purpose of this notebook is to couple the different use cases of the analysis mode of FAST-OAD-GA. A reference aircraft is chosen. At first the cabin length of this aircraft is increased, and then the wing span and longitudinal position are modified. The aim is to counterbalance the negative effects of the fuselage length increase with the modifications of the wings. Some quantities called "certification quantities" are defined below. The notebook proposes to observe the effect of the fuselage and of the wing modifications on these quantities.

List of the certification quantities that are used for the comparison between the aircrafts:
- Load Analysis of the wings (force, shear force and root bending moment distributions on the wing)
- Flight Envelope
- Static Margins (stick fixed and stick free)

The reference aircraft is preferably the output of a MultiDisciplinary Analysis. Please note that if the relative tolerance of the MDA is quite rough (rtol >= 1e3) then it is probable that the aircraft will undergo visible geometrical modifications when passed through the Geometry module. These modifications have nothing to do with the present analysis and to avoid them it is preferable to perform a MDA with a higher accuracy (rtol < 1e4). In that way the only visible modifications will be the one intended by the use case used and the rest of the aircraft geometry will remain untouched. The use of the dedicated notebook "MDA (adapted for analysis mode)" will prevent the user from encountering these issues.

The structure of this notebooks is described as follows:

**1. Initialization of the work environment.**

The user chooses the aircraft and the engine model he wants to work with.

**2. Definition of the fuselage modifications based on the reference architecture.**

The user specifies the cabin length increase he wants to apply, and can also choose the other parameters of the fuselage extension use case.

**3. Computation of the modified architecture.**

Like in the single use cases, the xml file of the modified architecture passes through all FAST-OAD-GA modules with generate_block_analysis. At the end of this section, the reference architecture file is saved under the name "OUTPUT_REF_FILE" while the extended fuselage architecture file is saved under the name "OUTPUT_MOD_FILE". These two files will form the base of the analysis of the wing span and wing position modifications. Since these files are saved in the output folder, the user can freely skip this part if is has already been performed and directly use the output files in the next sections. In that case the first section which initializes the files still has to be run.

**4. Analysis of the wing span increase.**

In this section the wing span of the modified architecture is changed. The aim is to study the effect of a wing span increase on the certification quantities. The user defines the different span changes he wants to make like in the wing span increase use case, and then the code returns the related computed aircrafts. A postprocessing cell compares the reference aircraft (OUTPUT_REF_FILE), the extended fuselage aircraft (OUTPUT_MOD_FILE) and the aircrafts with the wing span increased (an aircraft with +20% of wing span would be called output_span_1_2.xml).

**5. Analysis of the wing longitudinal position modification.**

In this section the wing longitudinal position of the modified architecture is changed. The aim is to study the effect of the wing longitudinal position modification on the certification quantities. The user defines the different wing position changes he wants to make and the code returns the related computed aircrafts. A postprocessing cell compares the reference aircraft (OUTPUT_REF_FILE), the extended fuselage aircraft (OUTPUT_MOD_FILE) and the aircrafts with the wing position modified (an aircraft with a wing moved back by 0.5 meters would be called output_wing_pos_delta_0_5.xml).

**6. Using fsolve to find the optimal wing position for the static margins**

The nonlinear solver fsolve is used in conjunction with the FAST-OAD-GA modules to determine the wing position for the modified architecture that results in a fixed static margin equal to the one of the reference aircraft.

Like in the tutorials, the structure of the notebook is based on data transfers between different folders :
- The data folder stocks the MDA outputs of the reference aircrafts. These files serve as reference files for the comparison.
- The workdir folder stocks the .xml files of the modified architectures passing through the modules of FAST-OAD-GA.
- The output folder stocks the final .xml files, which are ready for post-processing.

All the files of the different sections are saved in the output folder. They are ready to be used in the postprocessing notebook, if the user wants to have access to more visualization tools.

For a proper notebook execution, the use of the analysis_mode branch of FAST-GA-OAD is strongly recommended.

If the user wants to perform a new analysis after the notebook cells have already been run, for any change in the widget values, in the configuration files or in the aircraft modification parameters it is advised to restart the kernel to avoid unwelcomed inconsistencies.

## 1. Initialization of the work environment

### Choice of the aircraft

The available aircrafts are :

- Beechcraft Duchess 76
- Cirrus SR22
- TBM 930

The Beechcraft 76 is the default option.

In [1]:
from ipywidgets import widgets, Layout
from fastoad.io import VariableIO
from IPython.display import display, clear_output

liste_aircraft = widgets.Dropdown(
    options=[('Beechcraft Duchess 76', 'output_mda_beech.xml'), ('Cirrus SR22', 'output_mda_cirrus_sr22.xml'), ('TBM-930', 'output_mda_tbm930.xml')],
    value='output_mda_beech.xml',
    description='Reference Aircraft:',
    style={'description_width': 'initial'},
    layout=Layout(width='30%'),
)

# Initialization of the reference aircraft
ref_aircraft = 'output_mda_beech.xml'
print('The reference aircraft .xml file is '+ ref_aircraft)

def eventhandler(change):
    if (change.new):
        global ref_aircraft
        ref_aircraft = liste_aircraft.value
        print('The reference aircraft .xml file is '+ ref_aircraft)

liste_aircraft.observe(eventhandler, names='value')

display(liste_aircraft)

Failed to import module fastga.models.propulsion.fuel_propulsion.basicIC_engine.unitary_tests.test_openmdao_engine.py
Failed to import module fastga.models.performances.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.performances.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.aerodynamics.unitary_tests.test_functions.py
Failed to import module fastga.models.handling_qualities.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.aerodynamics.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.propulsion.fuel_propulsion.turboprop_simple.unitary_tests.test_openmdao_engine.py
Failed to import module fastga.models.geometry.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.weight.mass_breakdown.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.load_analysis.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.weight.mass_breakdown.unit

The reference aircraft .xml file is output_mda_beech.xml


Dropdown(description='Reference Aircraft:', layout=Layout(width='30%'), options=(('Beechcraft Duchess 76', 'ou…

### Choice of the engine model

The available models are :

- Basic IC Engine
- Basic Turboprop

The basic IC engine is the default option.

In [2]:
liste_engine = widgets.Dropdown(
    options=[('Basic IC Engine', "fastga.wrapper.propulsion.basicIC_engine"), ('Basic Turboprop', "fastga.wrapper.propulsion.basic_turboprop")],
    value="fastga.wrapper.propulsion.basicIC_engine",
    description='Propulsion Model:',
    style={'description_width': 'initial'},
    layout=Layout(width='30%'),
)

# Initialization of the engine model
engine_id = "fastga.wrapper.propulsion.basicIC_engine"
print('The engine model id chosen is '+ engine_id)

def eventhandler(change):
    if (change.new):
        global engine_id
        engine_id = liste_engine.value
        print('The engine model id chosen is '+ engine_id)

liste_engine.observe(eventhandler, names='value')

display(liste_engine)

The engine model id chosen is fastga.wrapper.propulsion.basicIC_engine


Dropdown(description='Propulsion Model:', layout=Layout(width='30%'), options=(('Basic IC Engine', 'fastga.wra…

### Preparation of files

The reference aircraft file "AIRCRAFT_REF_FILE" is a copy from the corresponding mda output file. The aircraft id depends on the choice made with the widget.

This file is then directly duplicated into "AIRCRAFT_MOD_FILE", which is the file that will receive the fuselage extension. So after the execution of this cell, both files are identical.

Please note that the engine_id widget allows for the user to skip the part where he has to manually write the engine's id in the parameters of the classes executed by generate_block_analysis. But for the weight/performance loop that uses the weight_loop.yml configuration file the user has to manually check that the right engine's id will be used for the weight and performance modules calculation. If the yml file is modified the changes will only take effect after the kernel is restarted.

In [3]:
import os.path as pth
import os
import openmdao.api as om
from fastoad import api as api_cs25
from fastga.command import api as api_cs23
import fastga
import logging
from fastoad.gui import VariableViewer
from fastoad.io import VariableIO
import shutil

# Define relative path.
current_path = os.getcwd()
DATA_FOLDER_PATH = pth.join(current_path, 'data')
WORK_FOLDER_PATH = pth.join(current_path, 'workdir')
OUTPUT_FOLDER_PATH = pth.join(current_path, 'output')

# Clear work folder.
shutil.rmtree(WORK_FOLDER_PATH, ignore_errors=True)
os.mkdir(WORK_FOLDER_PATH)

# For using more screen width.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Final file names. The .xml files in the work folder will be overwritten each time they pass through a module.
CONFIG_FILE = pth.join(DATA_FOLDER_PATH, 'weight_loop.yml')
WORK_CONFIG_FILE = pth.join(WORK_FOLDER_PATH, 'weight_loop.yml')
AIRCRAFT_REF_FILE = pth.join(WORK_FOLDER_PATH, 'geometry_ref.xml')
AIRCRAFT_MOD_FILE = pth.join(WORK_FOLDER_PATH, 'geometry_mod.xml')
WEIGHT_LOOP_FILE = pth.join(WORK_FOLDER_PATH, 'problem_inputs_weight_loop.xml')

# Copy the reference aircraft .xml file.
shutil.copy(pth.join(DATA_FOLDER_PATH, ref_aircraft), AIRCRAFT_REF_FILE)
shutil.copy(AIRCRAFT_REF_FILE, AIRCRAFT_MOD_FILE)
shutil.copy(CONFIG_FILE, WORK_CONFIG_FILE)

'C:\\Users\\Lucas\\FAST-OAD-GA\\FAST-GA\\src\\fastga\\notebooks\\Analysis Mode\\workdir\\weight_loop.yml'

## 2. Definition of the fuselage modifications based on the reference architecture

The different parameters the user has to choose in order to obtain the desired modified configuration are:

- **added_length [m]** : If 0, the length of a row of seats will be taken (this length is defined in the xml file).
- **added_section_x_ratio_front** : Float between 0 and 1 defining the percentage of the added section that will be placed ahead of the wing MAC 25%.
- **added_section_x_ratio_rear** : Float between 0 and 1 defining the percentage of the added section that will be placed at the back of the wing MAC 25%.
- **added_pax** : Integer stating the number of passengers added to the design mission. Take care not to exceed npax_max.
- **added_luggage [kg]** : Float stating the mass of luggage added to the design mission.

If a row of seats is added (added_length = 0), then the code will increase the maximum number of passengers. If a specified length is used (for example added_length = 0.5 m), then the code won't increase the maximum number of passengers. In this case the added section will be an empty space in the cabin.

The sum of added_section_x_ratio_front and of added_section_x_ratio_rear must be equal to 1.

There is no limit in the code for the increase in cabin length, but some modules could have problems running with an extreme value.
A cabin length reduction can also be theorically computed by specifying a negative length, but the consequences on the code have not been studied. 

In this particular notebook it would make sense to add a row of seats (added_length=0) and to add 2 passengers to the mission (filling the new row of seats).

In [ ]:
# Fuselage modifications
added_length = 0    # If 0, the length of a row of seats will be taken
added_section_x_ratio_front = 0
added_section_x_ratio_rear = 1
added_pax = 2     # Integer stating the number of passengers added in the design mission. It will be added to the npax_design of the xml, so take care not to exceed the new npax_max.
added_luggage = 10     # float stating the mass of luggage added in the design mission

fuselage_mod_parameters = [added_length, added_section_x_ratio_front, added_section_x_ratio_rear, added_pax, added_luggage]

## 3. Computation of the modified architecture

The process used in the following cell is the same as the one explained in the single use cases.

Here we execute first the modules that compute the fuselage modification and then we execute the classic FAST-OAD-GA modules to get the modifications resulting from the fuselage geometry changes.

The weight loop is a mda that loops on the weight / update mtow / performance modules in order to get converged MTOW and fuel mission.

The load analysis module is also executed for post-processing reasons (there is direct comparison with other aircrafts in this notebook). Please note that in this notebook the aircraft drag polars and the payload range are not computed like it is done in the single use cases notebooks. There is indeed no need in this analysis to display these postprocessing tools. If the user wants to compare the payload range diagrams of the aircrafts he has to first execute the class ComputePayloadRange with generate_block_analysis for each computed aircraft and then call the postprocessing function payload_range. An analog approach would be used for the aircraft drag polars.

It is advised to change the class parameters with caution as some modules require some of the quantities computed with these parameters on (for example the wing structural analysis requires the computation of the slipstream in the aerodynamic module, and the load factor class requires the mach interpolation computation).

The expected computational time is around 2 minutes. The aerodynamic module is the most demanding part of the analysis.

In [ ]:
from fastga.models.modify_config import ComputeConfigMod
from fastga.models.modify_config.update_XML import UpdateXML
from fastga.models.geometry import GeometryFixedTailDistance as Geometry
from fastga.models.aerodynamics.aerodynamics import Aerodynamics
from fastga.models.aerodynamics.load_factor import LoadFactor
from fastga.models.load_analysis.loads import Loads
from fastga.models.handling_qualities.handling_qualities import ComputeHandlingQualities


# Compute the modifications
compute_modify = api_cs23.generate_block_analysis(
        ComputeConfigMod(fuselage_mod=fuselage_mod_parameters),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output_mod = compute_modify({})

compute_update = api_cs23.generate_block_analysis(
        UpdateXML(fuselage_mod=fuselage_mod_parameters),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output_mod = compute_update({})

# GEOMETRY
compute_geometry_mod = api_cs23.generate_block_analysis(
        Geometry(propulsion_id=engine_id),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output_mod = compute_geometry_mod({})

# AERODYNAMICS
compute_aero_mod = api_cs23.generate_block_analysis(
        Aerodynamics(
            propulsion_id=engine_id,
            use_openvsp=True,
            compute_mach_interpolation=True,
            compute_slipstream_cruise=True,
#             wing_airfoil_file="naca43013_3.af"
        ),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output_mod = compute_aero_mod({})

# LOAD FACTOR (ComputeVN and speed identification)
compute_load_factor = api_cs23.generate_block_analysis(
        LoadFactor(propulsion_id=engine_id),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output = compute_load_factor({})

# WEIGHT / MTOW / PERFORMANCES loop

shutil.copy(AIRCRAFT_MOD_FILE, WEIGHT_LOOP_FILE)

eval_problem = api_cs25.evaluate_problem(WORK_CONFIG_FILE, overwrite=True)

OUTPUT_FILE = pth.join(WORK_FOLDER_PATH,'problem_outputs_weight_loop.xml')
shutil.copy(OUTPUT_FILE, AIRCRAFT_MOD_FILE)

# LOAD ANALYSIS
compute_loads_mod = api_cs23.generate_block_analysis(
        Loads(),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output_mod = compute_loads_mod({})

# HANDLING QUALITIES
compute_hq_mod = api_cs23.generate_block_analysis(
        ComputeHandlingQualities(
            propulsion_id=engine_id,
        ),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output_mod = compute_hq_mod({})

We save the reference architecture under the name OUTPUT_REF_FILE. The aircraft with extended fuselage is saved as OUTPUT_MOD_FILE. These two files are placed in the output folder and will be used in the next sections.

In [ ]:
OUTPUT_REF_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_ref_fuselage_be76.xml')
OUTPUT_MOD_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_mod_fuselage_be76.xml')
shutil.copy(AIRCRAFT_REF_FILE, OUTPUT_REF_FILE)
shutil.copy(AIRCRAFT_MOD_FILE, OUTPUT_MOD_FILE)

A plot of the geometry changes between the reference aircraft and the modified one.

In [ ]:
from fastga.utils.postprocessing.analysis_and_plots import aircraft_geometry_plot

fig = aircraft_geometry_plot(OUTPUT_REF_FILE, name='reference', plot_nacelle = False)
fig = aircraft_geometry_plot(OUTPUT_MOD_FILE, name='modified fuse', plot_nacelle = False, fig=fig)
fig.show()

In [ ]:
api_cs25.variable_viewer(AIRCRAFT_REF_FILE)

In [ ]:
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

## 4. Analysis of the wing span increase

In this section the wing span of the modified architecture (aircraft with extended fuselage) is changed. The aim is to study the effect of a wing span increase on the certification quantities. The user defines the different span changes he wants to make like in the wing span increase use case, and then the code returns the related computed aircrafts. A postprocessing cell compares the reference aircraft (OUTPUT_REF_FILE), the extended fuselage aircraft (OUTPUT_MOD_FILE) and the aircrafts with the wing span increased (an aircraft with +20% of wing span would be called output_span_1_2.xml).

Once the reference and modified output files are created this section can be independently run.

At first we call the output files and we create a temporary xml that will be used to compute the architectures with the increased span.

In [ ]:
OUTPUT_REF_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_ref_fuselage_be76.xml')
OUTPUT_MOD_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_mod_fuselage_be76.xml')
AIRCRAFT_SPAN_FILE = pth.join(WORK_FOLDER_PATH, 'aircraft_span_mod.xml')

In the next cell the user defines the wing span modifications. Just like in the wing span increase use case, the different parameters the user has to choose in order to obtain the desired modified configuration are:

- **span_length_multiplier** : value by which the wing span is multiplied. If 1.1, the span will be extended by 10% of its original value.
- **fixed_engine_position** : boolean stating if the engines stay at the same y position (y_ratio will have to be modified) or if their y_ratio is conserved.
- **fuel_added_part** : boolean stating if fuel tanks are added in the extended section of the wing. If True, only applies if y_ratio_tank_end = 1.0 in the reference architecture.

There is no limit in the code for the increase in span, but some modules could have problems running with an extreme value (such as two times the original span).
A wing span reduction can also be theorically computed by using a multiplier inferior to 1, but the consequences on the code have not been studied. 

In this particular notebook several aircrafts can be studied at once. The code will browse the span_multiplier_array and for each of its elements will compute a different aircraft. The approach used is the same as in the single use cases and in the third section of this notebook. At the end of each computation, the resulting file is saved and its path is stored in output_span_path_array. The postprocessing cell then browses this array and calls each file in the visualization tools. In that way the user only needs to fill the span_multiplier_array and to execute the cells.

In [ ]:
span_multiplier_array = [1.1, 1.2]
output_span_path_array = []

fixed_engine_position = True
fuel_added_part = False

In [ ]:
from fastga.models.modify_config import ComputeConfigMod
from fastga.models.modify_config.update_XML import UpdateXML
from fastga.models.geometry import GeometryFixedTailDistance as Geometry
from fastga.models.aerodynamics.aerodynamics import Aerodynamics
from fastga.models.aerodynamics.load_factor import LoadFactor
from fastga.models.load_analysis.loads import Loads
from fastga.models.handling_qualities.handling_qualities import ComputeHandlingQualities


for span_multiplier in span_multiplier_array:

    shutil.copy(OUTPUT_MOD_FILE, AIRCRAFT_SPAN_FILE)
    
    # Modify and update geometry
    compute_modify = api_cs23.generate_block_analysis(
            ComputeConfigMod(span_mod=[span_multiplier, fixed_engine_position, fuel_added_part]),
            [],
            str(AIRCRAFT_SPAN_FILE),
            True,
        )
    output = compute_modify({})

    compute_update = api_cs23.generate_block_analysis(
        UpdateXML(span_mod=[span_multiplier, fixed_engine_position, fuel_added_part]),
        [],
        str(AIRCRAFT_SPAN_FILE),
        True,
    )
    output = compute_update({})
    
    # GEOMETRY
    compute_geometry = api_cs23.generate_block_analysis(
            Geometry(propulsion_id=engine_id),
            [],
            str(AIRCRAFT_SPAN_FILE),
            True,
        )
    output = compute_geometry({})

    # AERODYNAMICS
    compute_aero = api_cs23.generate_block_analysis(
            Aerodynamics(
                propulsion_id=engine_id,
                use_openvsp=True,
                compute_mach_interpolation=True,
                compute_slipstream_cruise=True,
    #             wing_airfoil_file="naca43013_3.af"
            ),
            [],
            str(AIRCRAFT_SPAN_FILE),
            True,
        )
    output = compute_aero({})
    
    # LOAD FACTOR (ComputeVN and speed identification)
    compute_load_factor = api_cs23.generate_block_analysis(
            LoadFactor(propulsion_id=engine_id),
            [],
            str(AIRCRAFT_SPAN_FILE),
            True,
        )
    output = compute_load_factor({})

    # WEIGHT / MTOW / PERFORMANCES loop for modified aircraft
    shutil.copy(AIRCRAFT_SPAN_FILE, WEIGHT_LOOP_FILE)

    eval_problem = api_cs25.evaluate_problem(WORK_CONFIG_FILE, overwrite=True)

    OUTPUT_FILE = pth.join(WORK_FOLDER_PATH,'problem_outputs_weight_loop.xml')
    shutil.copy(OUTPUT_FILE, AIRCRAFT_SPAN_FILE)

    # LOAD ANALYSIS
    compute_loads = api_cs23.generate_block_analysis(
            Loads(),
            [],
            str(AIRCRAFT_SPAN_FILE),
            True,
        )
    output = compute_loads({})
    
    # HANDLING QUALITIES
    compute_hq = api_cs23.generate_block_analysis(
            ComputeHandlingQualities(
                propulsion_id=engine_id,
            ),
            [],
            str(AIRCRAFT_SPAN_FILE),
            True,
        )
    output = compute_hq({})
    
    
    output_file_name = 'output_span_' + str(span_multiplier).replace('.','_') + '.xml'
    shutil.copy(AIRCRAFT_SPAN_FILE, pth.join(OUTPUT_FOLDER_PATH, output_file_name))
    output_span_path_array.append(pth.join(OUTPUT_FOLDER_PATH, output_file_name))

### Post Processing

In [ ]:
from fastga.utils.postprocessing.load_analysis.analysis_and_plots_la import force_repartition_diagram, shear_diagram, rbm_diagram
from fastga.utils.postprocessing.analysis_and_plots import aircraft_geometry_plot
from fastga.utils.postprocessing.analysis_and_plots_vn import evolution_diagram


# Aircraft geometry plot
fig = aircraft_geometry_plot(OUTPUT_REF_FILE, name='reference aircraft', plot_nacelle = False)
fig = aircraft_geometry_plot(OUTPUT_MOD_FILE, name='extended fuselage', plot_nacelle = False, fig=fig)

for i in range(len(span_multiplier_array)):
    fig = aircraft_geometry_plot(output_span_path_array[i], name='wing span multiplied by ' + str(span_multiplier_array[i]), plot_nacelle = False, fig=fig)

fig.show()


# Force repartition diagram
fig = force_repartition_diagram(OUTPUT_REF_FILE, name='reference aircraft')
fig = force_repartition_diagram(OUTPUT_MOD_FILE, name='extended fuselage',fig=fig)

for i in range(len(span_multiplier_array)):
    fig = force_repartition_diagram(output_span_path_array[i], name='wing span multiplied by ' + str(span_multiplier_array[i]), fig=fig)

fig.show()


# Shear diagram
fig = shear_diagram(OUTPUT_REF_FILE, name='reference aircraft')
fig = shear_diagram(OUTPUT_MOD_FILE, name='extended fuselage',fig=fig)

for i in range(len(span_multiplier_array)):
    fig = shear_diagram(output_span_path_array[i], name='wing span multiplied by ' + str(span_multiplier_array[i]), fig=fig)

fig.show()


# Root bending moment diagram
fig = rbm_diagram(OUTPUT_REF_FILE, name='reference aircraft')
fig = rbm_diagram(OUTPUT_MOD_FILE, name='extended fuselage',fig=fig)

for i in range(len(span_multiplier_array)):
    fig = rbm_diagram(output_span_path_array[i], name='wing span multiplied by ' + str(span_multiplier_array[i]), fig=fig)

fig.show()


# Manoeuver Diagram
fig = evolution_diagram(OUTPUT_REF_FILE, name='reference aircraft')
fig = evolution_diagram(OUTPUT_MOD_FILE, name='extended fuselage', fig=fig)

for i in range(len(span_multiplier_array)):
    fig = evolution_diagram(output_span_path_array[i], name='wing span multiplied by ' + str(span_multiplier_array[i]), fig=fig)

fig.show()


# Static margins
variables_ref = VariableIO(OUTPUT_REF_FILE).read()
static_margin_fixed_ref = round(variables_ref["data:handling_qualities:stick_fixed_static_margin"].value[0],4)
static_margin_free_ref = round(variables_ref["data:handling_qualities:stick_free_static_margin"].value[0],4)
print("reference aircraft\t\t", "static margin stick fixed:", static_margin_fixed_ref, "\tstatic margin stick free:", static_margin_free_ref)

variables_mod = VariableIO(OUTPUT_MOD_FILE).read()
static_margin_fixed_mod = round(variables_mod["data:handling_qualities:stick_fixed_static_margin"].value[0],4)
static_margin_free_mod = round(variables_mod["data:handling_qualities:stick_free_static_margin"].value[0],4)
print("fuselage extended aircraft\t", "static margin stick fixed:", static_margin_fixed_mod, "\tstatic margin stick free:", static_margin_free_mod)

for i in range(len(span_multiplier_array)):
    variables = VariableIO(output_span_path_array[i]).read()
    static_margin_fixed = round(variables["data:handling_qualities:stick_fixed_static_margin"].value[0],4)
    static_margin_free = round(variables["data:handling_qualities:stick_free_static_margin"].value[0],4)
    print("wing span multiplied by", span_multiplier_array[i], "\t", "static margin stick fixed:", static_margin_fixed, "\tstatic margin stick free:", static_margin_free)

## 5. Analysis of the wing longitudinal position modification

In this section the wing longitudinal position of the modified architecture is changed. The aim is to study the effect of the wing longitudinal position modification on the certification quantities. The user defines the different wing position changes he wants to make and the code returns the related computed aircrafts. A postprocessing cell compares the reference aircraft (OUTPUT_REF_FILE), the extended fuselage aircraft (OUTPUT_MOD_FILE) and the aircrafts with the wing position modified (an aircraft with a wing moved back by 0.5 meters would be called output_wing_pos_delta_0_5.xml).

Once the reference and modified output files are created this section can be independently run.

At first we call the output files and we create a temporary xml that will be used to compute the architectures with the modified wing position.

In [ ]:
OUTPUT_REF_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_ref_fuselage_be76.xml')
OUTPUT_MOD_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_mod_fuselage_be76.xml')
AIRCRAFT_WING_POS_FILE = pth.join(WORK_FOLDER_PATH, 'aircraft_wing_pos_mod.xml')

In the next cell the user defines the wing modifications he wants to compute.

In order to change the wing longitudinal position there is no need to execute the classes from the modify_config module. Indeed the method generate_block_analysis offers the opportunity for the user to specify the value of inputs of the class he wants to execute. For this analysis we will directly modify the quantity that describes the distance between the nose of the aircraft and the x position of the MAC 25% of the wing (stored as data:geometry:wing:MAC:at25percent:x).

In FAST-OAD-GA the horizontal tail x position is computed by adding the distance between the nose of the aircraft and the x position of the MAC 25% of the wing, and the distance between the x position of the MAC 25% of the wing and the x position of the MAC 25% of the horizontal tail (stored as "data:geometry:horizontal_tail:MAC:at25percent:x:from_wingMAC25"). So if we want to modify the wing position without moving the horizontal tail, we have to modify also the distance between wing and tail.

The user defines a delta_wing quantity that will be added to the xml file wing position and substracted to the xml file distance between wing and tail. The sign convention here is that a positive delta_wing value corresponds to a shift of the wing towards the tail. A negative delta_wing would correspond to ta shift of the wing towards the nose of tha aircraft.

In this particular notebook several aircrafts can be studied at once. The code will browse the delta_wing_array and for each of its elements will compute a different aircraft. The approach used is the same as in the single use cases and in the third section of this notebook. At the end of each computation, the resulting file is saved and its path is stored in output_position_path_array. The postprocessing cell then browses this array and calls each file in the visualization tools. In that way the user only needs to fill the delta_wing_array and to execute the cells.

In [ ]:
variables_mod = VariableIO(OUTPUT_MOD_FILE).read()
wing_mac_25 = variables_mod["data:geometry:wing:MAC:at25percent:x"].value[0]
htp_25 = variables_mod["data:geometry:horizontal_tail:MAC:at25percent:x:from_wingMAC25"].value[0]

delta_wing_array = [0.327]
output_position_path_array = []

In [ ]:
from fastga.models.geometry import GeometryFixedTailDistance as Geometry
from fastga.models.aerodynamics.aerodynamics import Aerodynamics
from fastga.models.aerodynamics.load_factor import LoadFactor
from fastga.models.load_analysis.loads import Loads
from fastga.models.handling_qualities.handling_qualities import ComputeHandlingQualities


for delta_wing in delta_wing_array:
    
    shutil.copy(OUTPUT_MOD_FILE, AIRCRAFT_WING_POS_FILE)
    
    var_inputs = ["data:geometry:wing:MAC:at25percent:x", "data:geometry:horizontal_tail:MAC:at25percent:x:from_wingMAC25"]

    # GEOMETRY
    compute_geometry = api_cs23.generate_block_analysis(
            Geometry(propulsion_id=engine_id),
            var_inputs,
            str(AIRCRAFT_WING_POS_FILE),
            True,
        )
    
    inputs_dict = {"data:geometry:wing:MAC:at25percent:x":(wing_mac_25 + delta_wing, "m"), "data:geometry:horizontal_tail:MAC:at25percent:x:from_wingMAC25":(htp_25 - delta_wing, "m")}
    output = compute_geometry(inputs_dict)
    
    # AERODYNAMICS
    compute_aero = api_cs23.generate_block_analysis(
            Aerodynamics(
                propulsion_id=engine_id,
                use_openvsp=True,
                compute_mach_interpolation=True,
                compute_slipstream_cruise=False,
    #             wing_airfoil_file="naca43013_3.af"
            ),
            [],
            str(AIRCRAFT_WING_POS_FILE),
            True,
        )
    output = compute_aero({})
    
    # LOAD FACTOR (ComputeVN and speed identification)
    compute_load_factor = api_cs23.generate_block_analysis(
            LoadFactor(propulsion_id=engine_id),
            [],
            str(AIRCRAFT_WING_POS_FILE),
            True,
        )
    output = compute_load_factor({})

    # WEIGHT / MTOW / PERFORMANCES loop
    shutil.copy(AIRCRAFT_WING_POS_FILE, WEIGHT_LOOP_FILE)

    eval_problem = api_cs25.evaluate_problem(WORK_CONFIG_FILE, overwrite=True)

    OUTPUT_FILE = pth.join(WORK_FOLDER_PATH,'problem_outputs_weight_loop.xml')
    shutil.copy(OUTPUT_FILE, AIRCRAFT_WING_POS_FILE)
    
    # LOAD ANALYSIS
    compute_loads = api_cs23.generate_block_analysis(
            Loads(),
            [],
            str(AIRCRAFT_WING_POS_FILE),
            True,
        )
    output = compute_loads({})

    # HANDLING QUALITIES
    compute_hq = api_cs23.generate_block_analysis(
            ComputeHandlingQualities(
                propulsion_id=engine_id,
            ),
            [],
            str(AIRCRAFT_WING_POS_FILE),
            True,
        )
    output = compute_hq({})
    
    
    output_file_name = 'output_wing_pos_delta_' + str(delta_wing).replace('.','_') + '.xml'
    shutil.copy(AIRCRAFT_WING_POS_FILE, pth.join(OUTPUT_FOLDER_PATH, output_file_name))
    output_position_path_array.append(pth.join(OUTPUT_FOLDER_PATH, output_file_name))

### Post Processing

In [ ]:
from fastga.utils.postprocessing.load_analysis.analysis_and_plots_la import force_repartition_diagram, shear_diagram, rbm_diagram
from fastga.utils.postprocessing.analysis_and_plots import aircraft_geometry_plot
from fastga.utils.postprocessing.analysis_and_plots_vn import evolution_diagram


# Aircraft geometry plot
fig = aircraft_geometry_plot(OUTPUT_REF_FILE, name='reference aircraft', plot_nacelle = False)
fig = aircraft_geometry_plot(OUTPUT_MOD_FILE, name='extended fuselage', plot_nacelle = False, fig=fig)

for i in range(len(delta_wing_array)):
    fig = aircraft_geometry_plot(output_position_path_array[i], name='wing position moved by ' + str(delta_wing_array[i]), plot_nacelle = False, fig=fig)

fig.show()


# Force repartition diagram
fig = force_repartition_diagram(OUTPUT_REF_FILE, name='reference aircraft')
fig = force_repartition_diagram(OUTPUT_MOD_FILE, name='extended fuselage',fig=fig)

for i in range(len(delta_wing_array)):
    fig = force_repartition_diagram(output_position_path_array[i], name='wing position moved by ' + str(delta_wing_array[i]) + 'm', fig=fig)

fig.show()


# Shear diagram
fig = shear_diagram(OUTPUT_REF_FILE, name='reference aircraft')
fig = shear_diagram(OUTPUT_MOD_FILE, name='extended fuselage',fig=fig)

for i in range(len(delta_wing_array)):
    fig = shear_diagram(output_position_path_array[i], name='wing position moved by ' + str(delta_wing_array[i]) + 'm', fig=fig)

fig.show()


# Root bending moment diagram
fig = rbm_diagram(OUTPUT_REF_FILE, name='reference aircraft')
fig = rbm_diagram(OUTPUT_MOD_FILE, name='extended fuselage',fig=fig)

for i in range(len(delta_wing_array)):
    fig = rbm_diagram(output_position_path_array[i], name='wing position moved by ' + str(delta_wing_array[i]) + 'm', fig=fig)

fig.show()


# Manoeuver Diagram
fig = evolution_diagram(OUTPUT_REF_FILE, name='reference aircraft')
fig = evolution_diagram(OUTPUT_MOD_FILE, name='extended fuselage', fig=fig)

for i in range(len(delta_wing_array)):
    fig = evolution_diagram(output_position_path_array[i], name='wing position moved by ' + str(delta_wing_array[i]) + 'm', fig=fig)

fig.show()


# Static margins
variables_ref = VariableIO(OUTPUT_REF_FILE).read()
static_margin_fixed_ref = round(variables_ref["data:handling_qualities:stick_fixed_static_margin"].value[0],4)
static_margin_free_ref = round(variables_ref["data:handling_qualities:stick_free_static_margin"].value[0],4)
print("reference aircraft\t\t", "static margin stick fixed:", static_margin_fixed_ref, "\tstatic margin stick free:", static_margin_free_ref)

variables_mod = VariableIO(OUTPUT_MOD_FILE).read()
static_margin_fixed_mod = round(variables_mod["data:handling_qualities:stick_fixed_static_margin"].value[0],4)
static_margin_free_mod = round(variables_mod["data:handling_qualities:stick_free_static_margin"].value[0],4)
print("fuselage extended aircraft\t", "static margin stick fixed:", static_margin_fixed_mod, "\tstatic margin stick free:", static_margin_free_mod)

for i in range(len(delta_wing_array)):
    variables = VariableIO(output_position_path_array[i]).read()
    static_margin_fixed = round(variables["data:handling_qualities:stick_fixed_static_margin"].value[0],4)
    static_margin_free = round(variables["data:handling_qualities:stick_free_static_margin"].value[0],4)
    print("wing position moved by", delta_wing_array[i], 'm', "\t", "static margin stick fixed:", static_margin_fixed, "\tstatic margin stick free:", static_margin_free)

## 6. Using fsolve to find the optimal wing position for the static margins

For this section we want to find the optimal wing position for which the extended fuselage aircraft has the same fixed static margin as the reference aircraft. The next cell dislays the value of the static margins of the refernce and of the fuselage extended aircrafts.

We start by defining the function static_margins. This simple function computes a wing position shift by a delta_wing parameter which is its only input. The function then reads the static margin of the computed aircraft and compares it to the reference aircraft one. fsolve is used on this function as the objective is to minimize the difference between the reference static margin and the computed one. The xtol parameter of fsolve has been studied and the algorithm converges to a consistent value for xtol = 0.01. This results in 7 iterations (each iteration roughly requiring 3 minutes of computational time).

The 4 first fsolve iterations give results that seem identical and the code does not change the wing position. This is most likely the fsolve algorithm reacting to the fact that when the xml file goes through the FAST-OAD-GA modules all the quantities are updated.

The cell displays the found value of delta_wing when the algorithm has converged. The user can then manually generate the corresponding aircraft by putting the delta_wing value in input of the 5th section of this notebook.

Once the reference and modified output files are created this section can be independently run.

In [7]:
OUTPUT_REF_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_ref_fuselage_be76.xml')
OUTPUT_MOD_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_mod_fuselage_be76.xml')

variables_ref = VariableIO(OUTPUT_REF_FILE).read()
static_margin_fixed_ref = round(variables_ref["data:handling_qualities:stick_fixed_static_margin"].value[0],4)
static_margin_free_ref = round(variables_ref["data:handling_qualities:stick_free_static_margin"].value[0],4)
print("reference aircraft\t\t", "static margin stick fixed:", static_margin_fixed_ref, "\tstatic margin stick free:", static_margin_free_ref)

variables_mod = VariableIO(OUTPUT_MOD_FILE).read()
static_margin_fixed_mod = round(variables_mod["data:handling_qualities:stick_fixed_static_margin"].value[0],4)
static_margin_free_mod = round(variables_mod["data:handling_qualities:stick_free_static_margin"].value[0],4)
print("fuselage extended aircraft\t", "static margin stick fixed:", static_margin_fixed_mod, "\tstatic margin stick free:", static_margin_free_mod)

reference aircraft		 static margin stick fixed: 0.2498 	static margin stick free: 0.1405
fuselage extended aircraft	 static margin stick fixed: 0.154 	static margin stick free: 0.0308


In [8]:
from fastga.models.geometry import GeometryFixedTailDistance as Geometry
from fastga.models.aerodynamics.aerodynamics import Aerodynamics
from fastga.models.aerodynamics.load_factor import LoadFactor
from fastga.models.handling_qualities.handling_qualities import ComputeHandlingQualities

from scipy.optimize import fsolve

TEMP_FILE = pth.join(WORK_FOLDER_PATH, 'temp_file_static_margins.xml')
shutil.copy(OUTPUT_MOD_FILE, TEMP_FILE)

variables_mod = VariableIO(OUTPUT_MOD_FILE).read()

wing_mac_25 = variables_mod["data:geometry:wing:MAC:at25percent:x"].value[0]
htp_25 = variables_mod["data:geometry:horizontal_tail:MAC:at25percent:x:from_wingMAC25"].value[0]

delta_wing_initial = 0.3

def static_margins(
    delta_wing, file_formatter=None
):
    
    var_inputs = ["data:geometry:wing:MAC:at25percent:x",
                  "data:geometry:horizontal_tail:MAC:at25percent:x:from_wingMAC25"]

    # GEOMETRY
    compute_geometry = api_cs23.generate_block_analysis(
        Geometry(propulsion_id=engine_id),
        var_inputs,
        str(TEMP_FILE),
        True,
    )

    inputs_dict = {"data:geometry:wing:MAC:at25percent:x": (wing_mac_25 + delta_wing, "m"),
                   "data:geometry:horizontal_tail:MAC:at25percent:x:from_wingMAC25": (htp_25 - delta_wing, "m")}
    output = compute_geometry(inputs_dict)

    # AERODYNAMICS
    compute_aero = api_cs23.generate_block_analysis(
        Aerodynamics(
            propulsion_id=engine_id,
            use_openvsp=True,
            compute_mach_interpolation=True,
            compute_slipstream_cruise=False,
        ),
        [],
        str(TEMP_FILE),
        True,
    )
    output = compute_aero({})
    
    # LOAD FACTOR (ComputeVN and speed identification)
    compute_load_factor = api_cs23.generate_block_analysis(
            LoadFactor(
                propulsion_id=engine_id,
            ),
            [],
            str(TEMP_FILE),
            True,
        )
    output = compute_load_factor({})
    
    # WEIGHT / MTOW / PERFORMANCES loop
    shutil.copy(TEMP_FILE, WEIGHT_LOOP_FILE)

    eval_problem = api_cs25.evaluate_problem(WORK_CONFIG_FILE, overwrite=True)

    OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_weight_loop.xml')
    shutil.copy(OUTPUT_FILE, TEMP_FILE)

    # HANDLING QUALITIES
    compute_hq = api_cs23.generate_block_analysis(
        ComputeHandlingQualities(
            propulsion_id=engine_id,
        ),
        [],
        str(TEMP_FILE),
        True,
    )
    output = compute_hq({})

    variables_computed = VariableIO(TEMP_FILE, file_formatter).read()

    static_margin_computed = round(variables_computed["data:handling_qualities:stick_fixed_static_margin"].value[0],4)
    
    print("delta wing of current iteration [m] : " + str(delta_wing))
    print("resulting static margin : " + str(static_margin_computed))

    return static_margin_computed - static_margin_fixed_ref

# delta = static_margins(0.3)
# print(delta)
delta_wing_output, dic, _, _ = fsolve(static_margins, delta_wing_initial, xtol=0.01, full_output=True)

print("optimal delta wing : " + str(delta_wing_output[0]))
print(dic["nfev"])

Area ratio value outside of the range in Roskam's book k_vh, value clipped2.6162514994300965
min value of Roskam's book is 8.164931618692696e-05
max value of Roskam's book is 1.9750765462339261
Area ratio value outside of the range in Roskam's book k_vh, value clipped2.6162514994300965
min value of Roskam's book is 8.164931618692696e-05
max value of Roskam's book is 1.9750765462339261


NL: NLBGS 1 ; 180005857 1
NL: NLBGS 2 ; 10778.7827 5.98801777e-05
NL: NLBGS 3 ; 2528.73629 1.40480778e-05
NL: NLBGS 4 ; 625.441446 3.47456163e-06
NL: NLBGS 5 ; 156.416956 8.6895481e-07
NL: NLBGS Converged
0.22272638914815168
[0.3]


Area ratio value outside of the range in Roskam's book k_vh, value clipped2.6162514994300965
min value of Roskam's book is 8.164931618692696e-05
max value of Roskam's book is 1.9750765462339261
Area ratio value outside of the range in Roskam's book k_vh, value clipped2.6162514994300965
min value of Roskam's book is 8.164931618692696e-05
max value of Roskam's book is 1.9750765462339261


NL: NLBGS 1 ; 180005858 1
NL: NLBGS 2 ; 10778.3891 5.98779908e-05
NL: NLBGS 3 ; 2528.41877 1.40463139e-05
NL: NLBGS 4 ; 625.300654 3.47377948e-06
NL: NLBGS 5 ; 156.375261 8.68723183e-07
NL: NLBGS Converged
0.22273720120885765
[0.3]


Area ratio value outside of the range in Roskam's book k_vh, value clipped2.6162514994300965
min value of Roskam's book is 8.164931618692696e-05
max value of Roskam's book is 1.9750765462339261
Area ratio value outside of the range in Roskam's book k_vh, value clipped2.6162514994300965
min value of Roskam's book is 8.164931618692696e-05
max value of Roskam's book is 1.9750765462339261


NL: NLBGS 1 ; 180005858 1
NL: NLBGS 2 ; 10778.3953 5.98780252e-05
NL: NLBGS 3 ; 2528.4237 1.40463413e-05
NL: NLBGS 4 ; 625.302878 3.47379183e-06
NL: NLBGS 5 ; 156.375917 8.68726822e-07
NL: NLBGS Converged
0.2227370318099255
[0.3]


Area ratio value outside of the range in Roskam's book k_vh, value clipped2.6162514994300965
min value of Roskam's book is 8.164931618692696e-05
max value of Roskam's book is 1.9750765462339261
Area ratio value outside of the range in Roskam's book k_vh, value clipped2.6162514994300965
min value of Roskam's book is 8.164931618692696e-05
max value of Roskam's book is 1.9750765462339261


NL: NLBGS 1 ; 180005858 1
NL: NLBGS 2 ; 10778.3952 5.98780247e-05
NL: NLBGS 3 ; 2528.4236 1.40463407e-05
NL: NLBGS 4 ; 625.302854 3.4737917e-06
NL: NLBGS 5 ; 156.375907 8.68726769e-07
NL: NLBGS Converged
0.222737035455144
[0.3]


Area ratio value outside of the range in Roskam's book k_vh, value clipped2.6162514994300965
min value of Roskam's book is 8.164931618692696e-05
max value of Roskam's book is 1.9750765462339261
Area ratio value outside of the range in Roskam's book k_vh, value clipped2.6162514994300965
min value of Roskam's book is 8.164931618692696e-05
max value of Roskam's book is 1.9750765462339261


NL: NLBGS 1 ; 180005858 1
NL: NLBGS 2 ; 10801.9434 6.00088438e-05
NL: NLBGS 3 ; 2546.86037 1.41487638e-05
NL: NLBGS 4 ; 633.546802 3.51958992e-06
NL: NLBGS 5 ; 153.063408 8.50324593e-07
NL: NLBGS Converged
0.22999066062929668
[0.33318893]
delta_wing opti: 0.3331889281498599
3
